In [46]:
from gurobipy import Model, quicksum, GRB

In [47]:
Cust = set(range(1,3))
DC = Cust.copy() 
Route = {(i,j) for i in DC for j in Cust}
CT = {1,2}

T = 2
Period = list(range(1, T+1))

In [48]:
delivery = {(i,j):5 for i,j in Route}
dc_run = {(i,ct):100*ct for i in DC for ct in CT}

demand = {(j,t):10 for j in Cust for t in Period}

In [49]:
model = Model()

In [56]:
X, x, y = {}, {}, {}

for i in DC:
    for t in Period:
        X[i,t] = model.addVar(vtype="C", name=f'X[{i},{t}]')

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for i in DC:
    for ct in CT:
        for t in Period:
            y[i,ct,t] = model.addVar(vtype="B", name=f'y[{i},{ct},{t}]')

model.update()

In [59]:
Cust_Demand_Cons, DC_Running_Cons, CT_Only_Cons, DC_Flow_Cons = {}, {}, {}, {}

for j in Cust:
    for t in Period:
        Cust_Demand_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in DC)
            ==
            demand[j,t]
        )

for i,j in Route:
    for t in Period:
        DC_Running_Cons[i,j] = model.addConstr(
            x[i,j,t]
            <=
            quicksum(demand[j,t] for t in Period) * quicksum(y[i,ct,t] for ct in CT)
        )

for i in DC:
    for t in Period:
        CT_Only_Cons[i,t] = model.addConstr(
            quicksum(y[i,ct,t] for ct in CT)
            <=
            1
        )

for i in DC:
    for t in Period:
        DC_Flow_Cons = model.addConstr(
            X[i,t]
            ==
            quicksum(x[i,j,t] for j in Cust)
        )
    
model.update()

In [60]:
model.setObjective(
    quicksum(delivery[i,j] * x[i,j,t] for i,j in Route for t in Period) +
    quicksum(dc_run[i,ct] * y[i,ct,t] for i in DC for ct in CT for t in Period)
    ,GRB.MINIMIZE
)

model.update()

In [61]:
model.optimize()

Optimize a model with 68 rows, 56 columns and 172 nonzeros
Variable types: 32 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]

MIP start produced solution with objective 400 (0.01s)
Loaded MIP start with objective 400

Presolve removed 68 rows and 56 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 400 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+02, best bound 4.000000000000e+02, gap 0.0000%


In [62]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

x[1,2,1] 10.0
x[1,2,2] 10.0
x[1,1,1] 10.0
x[1,1,2] 10.0
y[1,2,1] 1.0
y[1,2,2] 1.0
y[2,2,1] 1.0
y[2,2,2] 1.0
X[2,1] 20.0
X[2,2] 20.0
x[2,1,1] 10.0
x[2,1,2] 10.0
x[2,2,1] 10.0
x[2,2,2] 10.0
y[2,1,1] 1.0
y[2,1,2] 1.0
